<a href="https://colab.research.google.com/github/fdossi/EDSMapPlotter/blob/main/EDSMapPlotter_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title 1. Setup and Configuration
# @markdown Run this cell to load libraries and define the processing function.

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import files
import shutil

# Create output directory
output_folder = "/content/processed_maps"
os.makedirs(output_folder, exist_ok=True)

def gerar_eds_map_colab(filename, cmap_name="Blues"):
    try:
        # Read the file directly
        df = pd.read_csv(filename, header=None)
        df_numeric = df.apply(pd.to_numeric, errors="coerce")
        data = df_numeric.to_numpy(dtype=float)

        # Basic validation
        if data.size == 0 or np.all(np.isnan(data)):
            print(f"⚠️ Skipped {filename}: Invalid data")
            return

        # Name parsing
        n0 = os.path.splitext(filename)[0]
        parts = n0.split("_")
        if len(parts) > 1:
            elemento = parts[-1]
            prefix = "_".join(parts[:-1])
            out_name = f"{prefix}_edsmap_{elemento}.png"
        else:
            elemento = n0
            out_name = f"edsmap_{elemento}.png"

        save_path = os.path.join(output_folder, out_name)

        # Plotting
        fig, ax = plt.subplots(figsize=(8, 6))
        hm = sns.heatmap(data, cmap=cmap_name, cbar_kws={"label": "Intensidade"}, xticklabels=False, yticklabels=False)
        ax.set_title(f"EDS-Map - {elemento}", fontsize=28)
        cb = hm.collections[0].colorbar
        cb.ax.tick_params(labelsize=18)
        cb.set_label("Intensidade", fontsize=22)

        plt.tight_layout()

        # Save and Show
        fig.savefig(save_path, dpi=300, bbox_inches="tight")
        print(f"✅ Processed: {out_name}")
        plt.show() # Display inline in Colab
        plt.close(fig)

    except Exception as e:
        print(f"❌ Error processing {filename}: {str(e)}")

print("System ready. Move to the next cell.")

In [ ]:
# @title 2. Upload and Process
# @markdown Select your color map and run this cell to upload your CSV files.

# Dropdown menu for Colab
Colormap = "viridis" #@param ["Blues", "viridis", "magma", "inferno", "plasma", "cividis", "Greys", "Reds", "Greens", "Oranges", "Purples", "turbo", "Spectral", "coolwarm", "seismic"]

print("⬆️ Please upload your CSV files below:")
uploaded = files.upload()

print("\n--- Processing ---")
for filename in uploaded.keys():
    gerar_eds_map_colab(filename, Colormap)

# Zip and download results
shutil.make_archive("/content/eds_maps_result", 'zip', output_folder)
print("\n🎉 Done! Downloading results...")
files.download("/content/eds_maps_result.zip")